# Data preprocessing and augmentation

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import glob
import pandas as pd
import cv2
import os
from shutil import copyfile
import numpy as np

# Libraries required for zooming
from scipy.ndimage import rotate
#from google.colab.patches import cv2_imshow
import numpy as np
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

# Patchset to allow cv2.imshow
from google.colab.patches import cv2_imshow
!pip install image_slicer
import image_slicer

In [0]:
# training_images_path = "/content/drive/My Drive/NN-ProjectC/Project_C1/Training"

# annotated_data = "/content/drive/My Drive/NN-ProjectC/Project_C1/train_files.csv"

# annotated_data = pd.read_csv(annotated_data, header=0)

In [0]:
# def get_annotation(label_data, file):
#     for rec in label_data.iterrows():
#         if rec[1]['file_name'] == file:
#             return rec[1]["annotation"]

In [0]:
# for image in os.listdir(training_images_path):
#     if image.endswith(".jpg"):
#       annotation = get_annotation(annotated_data, image)
#       new_path = os.path.join(training_images_path, "classified_images", str(annotation))
#       if not os.path.exists(new_path):
#           os.makedirs(new_path)
#       copyfile(os.path.join(training_images_path, image) , os.path.join(new_path, image))

In [0]:
# Checking number of images per folder
path = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Data"
for folder in os.listdir(path):
  print ("Class", folder, ":", len(os.listdir(os.path.join(path, folder))))


Class 4 : 76
Class 1 : 157
Class 3 : 116
Class 2 : 113
Class 0 : 437


In [0]:
def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

In [0]:
def increase_contrast(image):
    return cv2.convertScaleAbs(image, alpha = 1, beta = 50)

In [0]:
def decrease_brightness(image):
    return cv2.convertScaleAbs(image, alpha = 0.7, beta = 1)

In [0]:
def blur(image):
    return [cv2.blur(image,(3,3))]

In [0]:
def rotate_zoom(image):
    rot = rotate(image, 20, reshape=False)
    return clipped_zoom(rot, 2)

In [0]:
def rotate_zoom2(image):
    rot = rotate(image, 5, reshape=False)
    return clipped_zoom(rot, 2)

In [0]:
def sharpen(image):
  kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
  return cv2.filter2D(image, -1, kernel)

In [0]:
def slice(path, dest_path):
  f = path.split("/")
  f_name = f[-1].split(".")[-2]  

  tiles = image_slicer.slice(path, 4, save=False)

  image_slicer.save_tiles(tiles, directory = dest_path, prefix = f_name, format = 'jpeg')

In [0]:
# categories = ["0", "1", "2", "3", "4"]

In [0]:
# DATA_PATH = "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/classified_images"

Augmenting all the images with above transformations


In [0]:
# for category in categories:
    
#     for file in os.listdir(DATA_PATH + category):
        
#         image = cv2.imread(DATA_PATH + category + "/" + file)
        
#         f_name = file.split(".")[:-1][0]
        
#         print(f_name)
        
#         cv2.imwrite(DATA_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
  
#         cv2.imwrite(DATA_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
  
#         cv2.imwrite(DATA_PATH + category + "/" + f_name + "_c.jpg", blur(image))
  
#         cv2.imwrite(DATA_PATH + category + "/" + f_name + "_d.jpg", rotate_zoom(image))                                                        
    
#     print("Category: {} done".format(category))

# ! rm -rf "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/2"
# ! rm -rf "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/0"
# ! rm -rf "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/1"
# ! rm -rf "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/3"
# ! rm -rf "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/4"

!mkdir "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/0"
!mkdir "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/1"
!mkdir "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/2"
!mkdir "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/3"
!mkdir "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/4"

Keeping Class 0 as is

Augmenting Class 1: Each image x 4 transformations



In [0]:
# Copied the directory 0 as it is
from distutils.dir_util import copy_tree

# Copy subdirectory example
src = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Data/0/"
dst = "/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/0/"

for image in os.listdir(src):
    print (dst + image)
    copyfile(src + image , dst + image)

In [0]:
DATA_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Data/"
AUG_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Augmented_Data/"

category = "1"
counter = 0
for f in os.listdir(DATA_PATH+category):  
      image = cv2.imread(DATA_PATH  + category + "/" + f)
      f_name = f.split(".")[:-1][0]
      if counter == 0:  
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))
      elif counter == 1:
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_d.jpg", rotate_zoom(image))
      elif counter == 2:
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))
      elif counter == 3:
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_d.jpg", rotate_zoom(image)) 
      
      if counter == 2:
        counter = 0
      else:
        counter += 1
      copyfile(DATA_PATH  + category + "/" + f , AUG_PATH + category + "/" + f)
      print("Done with ", f)

In [0]:
category = "3"
counter = 0
for f in os.listdir(DATA_PATH+category):  
      image = cv2.imread(DATA_PATH  + category + "/" + f)
      f_name = f.split(".")[:-1][0]
      if counter == 0:  
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_d.jpg", rotate_zoom(image))
      elif counter == 1:
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_d.jpg", rotate_zoom(image))
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))
      if counter == 2:
        counter = 0
      else:
        counter += 1
      copyfile(DATA_PATH  + category + "/" + f , AUG_PATH + category + "/" + f)
      # print("Done with ", f)

category = "4"
for f in os.listdir(DATA_PATH+category):  
      image = cv2.imread(DATA_PATH  + category + "/" + f)
      f_name = f.split(".")[:-1][0]

      cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
      cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
      cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))
      cv2.imwrite(AUG_PATH + category + "/" + f_name + "_d.jpg", rotate_zoom(image)) 
      copyfile(DATA_PATH  + category + "/" + f , AUG_PATH + category + "/" + f)
      # print("Done with ", f)

category = "2"
counter = 0
for f in os.listdir(DATA_PATH+category):  
      image = cv2.imread(DATA_PATH  + category + "/" + f)
      f_name = f.split(".")[:-1][0]
      if counter == 0:  
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_d.jpg", rotate_zoom(image))
      elif counter == 1:
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_d.jpg", rotate_zoom(image))
        cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))
      # elif counter == 2:
      #   cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))
      # elif counter == 3:
      #   cv2.imwrite(AUG_PATH + category + "/" + f_name + "_d.jpg", rotate_zoom(image)) 
      
      if counter == 2:
        counter = 0
      else:
        counter += 1
      copyfile(DATA_PATH  + category + "/" + f , AUG_PATH + category + "/" + f)
      # print("Done with ", f)


In [0]:
# for image in os.listdir("/content/drive/My Drive/NN-ProjectC/Project_C1/Training/classified_images/1"):
#   print (image)
#   if "_" in image:
#     print (image)
#     os.remove("/content/drive/My Drive/NN-ProjectC/Project_C1/Training/classified_images/1/"+image)
# print (len(os.listdir("/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/1/")))
print (len(os.listdir("/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/0/")))
print (len(os.listdir("/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/1/")))
print (len(os.listdir("/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/4/")))
print (len(os.listdir("/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/2/")))
print (len(os.listdir("/content/drive/My Drive/NN-ProjectC/Project_C1/Training/augmented_images/3/")))



437
360
380
341
350


## Class 4

In [0]:
DATA_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Data/"
AUG_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Augmented_Data/"

In [0]:
category = "4"

In [0]:
for f in os.listdir(DATA_PATH +  category):    
  slice(DATA_PATH +  category + "/" + f, DATA_PATH + category)  

In [0]:
for f in os.listdir(DATA_PATH + category):  

  image = cv2.imread(DATA_PATH  + category + "/" + f)
  image = cv2.resize(image, (224, 224))
  f_name = f.split(".")[:-1][0]

  cv2.imwrite(AUG_PATH + category + "/" + f_name + ".jpg", image)
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))   
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))    
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))        

Class 3

In [0]:
DATA_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Data/"
AUG_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Augmented_Data/"

In [0]:
category = "3"

In [0]:
for f in os.listdir(DATA_PATH +  category):    
  slice(DATA_PATH +  category + "/" + f, DATA_PATH + category)  

In [0]:
i = 0

for f in os.listdir(DATA_PATH + category):  
  
  image = cv2.imread(DATA_PATH  + category + "/" + f)
  image = cv2.resize(image, (224, 224))
  f_name = f.split(".")[:-1][0]

  cv2.imwrite(AUG_PATH + category + "/" + f_name + ".jpg", image)

  if i % 4 == 0:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))   
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))        

  elif i % 4 == 1:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))       
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 2:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 3:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))   
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  
    

  i = i + 1

# Class 2

In [0]:
DATA_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Data/"
AUG_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Augmented_Data/"

In [0]:
category = "2"

In [0]:
for f in os.listdir(DATA_PATH +  category):    
  slice(DATA_PATH +  category + "/" + f, DATA_PATH + category)  

In [0]:
i = 0

for f in os.listdir(DATA_PATH + category):  
  
  image = cv2.imread(DATA_PATH  + category + "/" + f)
  image = cv2.resize(image, (224, 224))
  f_name = f.split(".")[:-1][0]

  cv2.imwrite(AUG_PATH + category + "/" + f_name + ".jpg", image)

  if i % 4 == 0:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))   
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))        

  elif i % 4 == 1:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))       
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 2:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 3:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))   
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  
    

  i = i + 1

## Class 1

In [0]:
DATA_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Data/"
AUG_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Augmented_Data/"

In [0]:
category = "1"

In [0]:
for f in os.listdir(DATA_PATH +  category):    
  slice(DATA_PATH +  category + "/" + f, DATA_PATH + category)  

In [0]:
i = 0

for f in os.listdir(DATA_PATH + category):  
  
  image = cv2.imread(DATA_PATH  + category + "/" + f)
  image = cv2.resize(image, (224, 224))
  f_name = f.split(".")[:-1][0]

  cv2.imwrite(AUG_PATH + category + "/" + f_name + ".jpg", image)

  if i % 4 == 0:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))       

  elif i % 4 == 1:
       
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))       
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 2:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))        

  elif i % 4 == 3:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))        
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  
    

  i = i + 1

# Class 0


In [0]:
DATA_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Data/"
AUG_PATH = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Augmented_Data/"

In [0]:
category = "0"

In [0]:
i = 0
for f in os.listdir(DATA_PATH +  category):    
  if i % 8 == 0:
    slice(DATA_PATH +  category + "/" + f, DATA_PATH + category)  
  i = i + 1

In [0]:
i = 0

for f in os.listdir(DATA_PATH + category):  
  
  image = cv2.imread(DATA_PATH  + category + "/" + f)
  image = cv2.resize(image, (224, 224))
  f_name = f.split(".")[:-1][0]

  cv2.imwrite(AUG_PATH + category + "/" + f_name + ".jpg", image)

  if i % 4 == 0:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))       
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 1:
       
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))       
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 2:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))        
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 3:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))        
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))        

  i = i + 1

In [0]:
path = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Data/"
for folder in os.listdir(path):
  print ("Class", folder, ":", len(os.listdir(os.path.join(path, folder))))

Class 4 : 380
Class 1 : 785
Class 3 : 580
Class 2 : 565
Class 0 : 657


In [0]:
path = "/content/drive/My Drive/NN-ProjectC/Version-2/Training/Augmented_Data/"
for folder in os.listdir(path):
  print ("Class", folder, ":", len(os.listdir(os.path.join(path, folder))))

Class 0 : 2464
Class 2 : 2825
Class 3 : 2900
Class 4 : 2280
Class 1 : 2355
